In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

arquivo_tweets = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/coronaVirus.csv')
print('arquivos lidos')

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-758cdbe2-280f-4970-ae99-a9f2b8c91d97;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 458ms :: artifacts dl 12ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

arquivos lidos


In [2]:
#Gerando tabela p/ word Cloud

import pyspark.sql.functions as f

# count_palavras_1=arquivo_tweets.withColumn('palavra', f.explode(f.split(f.col('Tweet'), ' ')))\
#     .groupBy('palavra')\
#     .count()\
#     .sort('count', ascending=False)\
#     .filter(col('palavra').isin('',
#                                 'corona',
#                                 'virus',
#                                 'corona',
#                                 'Corona',
#                                 'Virus',
#                                 'covid',
#                                 'covid19',
#                                 'Covid',
#                                 'covid19',
#                                 'covid-19')==False)

count_palavras_1=arquivo_tweets\
    .withColumn('palavra', f.explode(f.split(f.col('Tweet'), ' ')))\
    .withColumn('ajustado', f.when(col('palavra').isin('corona',
                                'virus',
                                'corona',
                                'Corona',
                                'Covid-19',
                                'Coronga',
                                'covid',
                                'covid19',
                                'Covid',
                                'covid19',
                                'covid-19',
                                'covid2019'),'CORONA')\
                            .otherwise(col('palavra')))\
    .groupBy('ajustado')\
    .count()\
    .sort('count', ascending=False)\
    .filter(col('ajustado').isin('')==False)\
    .withColumn('data_inserção',f.current_timestamp())

In [3]:
#criando Data Fream
base_tweets_brutos=arquivo_tweets.select(
    col('Tweet'),
    col('Date'),
    col('Source')
).withColumn('id_tweet',f.monotonically_increasing_id())

In [4]:
#Salvando S3 estado_IDH
count_palavras_1.write.mode('append').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-final/count_palavras_tweets.csv/')

#Salvando S3 tweets brutos
base_tweets_brutos.write.mode('append').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-final/base_tweets_brutos.csv/')

22/06/07 00:15:31 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 00:15:34 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 00:15:36 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 00:15:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [10]:
#Salvando Azure - WordCloud
print("inserindo azure")
# count_palavras_1.write \
# .format('com.microsoft.sqlserver.jdbc.spark') \
# .mode('overwrite') \
# .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
# .option('url', 'jdbc:sqlserver://servidor-grupo04.database.windows.net;databaseName=grupo04;') \
# .option('dbtable', 'table_count_tweets') \
# .option('user', 'urubu100') \
# .option('password', 'Urubu1@@') \
# .save()
# print("Tabela por caso salva")

base_tweets_brutos.write \
.format('com.microsoft.sqlserver.jdbc.spark') \
.mode('overwrite') \
.option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
.option('url', 'jdbc:sqlserver://yuiti.database.windows.net;databaseName=grupo04;') \
.option('dbtable', 'table_raw_tweets') \
.option('user', 'yuiti') \
.option('password', 'Thiago123') \
.save()

# #Salvando Azure - tweets brutos
# print("inserindo azure")
# base_tweets_brutos.write \
# .format('com.microsoft.sqlserver.jdbc.spark') \
# .mode('overwrite') \
# .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
# .option('url', 'jdbc:sqlserver://servidor-grupo04.database.windows.net;databaseName=grupo04;') \
# .option('dbtable', 'table_raw_tweets') \
# .option('user', 'urubu100') \
# .option('password', 'Urubu1@@') \
# .save()
# print("Tabela por caso salva")


inserindo azure
